# Refleksja

## Struktura pliku .class, dostęp do informacji o typie, polach i metodach
## Dynamiczne ładowanie klas

<br/>

## dr inż. Aleksander Smywiński-Pohl

## apohllo@o2.pl

## http://apohllo.pl/dydaktyka/programowanie-obiektowe

<img src="img/monster_mirror.jpg"/>

<img src="img/java_log.png" style="height: 700px"/>

In [ ]:
class DummyIterator implements Iterator {
    private int index = 0;
    
    @Override
    public boolean hasNext(){
        return false;
    }
    
    @Override
    public Object next(){
        return null;
    }
}

In [ ]:
javac DummyIterator.java

In [ ]:
cat DummyIterator.class

```
����4
	indexI<init>()VCodeLineNumberTablehasNext()Znext()Ljava/lang/Object;
SourceFileDummyIterator.java
                             	
DummyIteratorjava/lang/Objectjava/util/Iterator      
&
*�*��



�
       
�
%        
```

In [ ]:
javap DummyIterator

In [ ]:
Compiled from "DummyIterator.java"
class DummyIterator implements java.util.Iterator {
  DummyIterator();
  public boolean hasNext();
  public java.lang.Object next();
}

# Informacje o typie

In [ ]:
Class<?> klass = null;
klass = int.class;
klass = Integer.class;
klass = "Ala ma kota".getClass();
klass = new String[0].getClass();
klass = new DummyIterator().getClass();

# Weryfikacja przynależności do typu

In [ ]:
Object text = "To kot, a to Ala"
if(text.getClass().equals(String.class)){
    String textAsString = (String) text;
}

In [ ]:
Object iterator = new DummyIterator();
if(iterator.getClass.equals(Iterator.class)) {
    out.println("Obiekt jest iteratorem")
}

In [ ]:
if(iterator instanceof Iterator) {
    for(Object element : (Iterator) iterator) {
        out.println(element)
    }
}

# Klasa nadrzędna

In [ ]:
Class<?> klass = null;
klass = new DummyIterator().getClass();
klass = klass.getSuperclass();

# Wspólna klasa nadrzędna

In [ ]:
class A {}
class B1 extends A {}
class C1 extends B1 {}
class B2 extends A {}
class C2 extends B2 {}

In [ ]:
List<Class<?>> ancestors(Class<?> klass){
    List<Class<?>> result = new LinkedList<>();
    while(klass != Object.class){
        result.add(klass);
        klass = klass.getSuperclass();
    }
    result.add(Object.class);
    Collections.reverse(result);
    return result;
}

In [ ]:
ancestors(C2.class);

In [ ]:
Class<?> commonAncestor(Class<?> aClass, Class<?> bClass){
    List<Class<?>> aAncestors = ancestors(aClass);
    List<Class<?>> bAncestors = ancestors(bClass);
    for(int i = 0; i < aAncestors.size() && i < bAncestors.size(); i++){
        if(aAncestors.get(i) != bAncestors.get(i)){
            return aAncestors.get(i-1);
        }
    }
    return aAncestors.get(aAncestors.size()-1);
}

In [ ]:
commonAncestor(C1.class, C2.class);

<img src="img/cat1.jpg"/>

# Dynamiczne ładowanie klas

In [ ]:
interface IVehicle {
    void go();
}

In [ ]:
class Car implements IVehicle {
    public void go(){
        System.out.println("Brum, brum");
    }
}

In [ ]:
class Airplane implements IVehicle {
    public void go(){
        System.out.println("Bzzzzzz");
    }
}

In [ ]:
class Rocket implements IVehicle {
    public void go(){
        System.out.println("BUUUUUUUUUUUM");
    }
}

In [ ]:
class VehicleSystem {
    public static void main(String[] args){
        List<IVehicle> vehicles = new LinkedList<>();
        for(String vehicleName : args){
            try {
                Class<IVehicle> klass = 
                    (Class<IVehicle>) Class.forName(vehicleName);
                vehicles.add(klass.newInstance());
            } catch(ClassNotFoundException ex) {
                System.out.println("The class " + vehicleName +  
                                   " has not been found. ");
                return;
            } catch(InstantiationException | IllegalAccessException ex) {
                System.out.println("The class " + vehicleName +  
                                   " cannot be instantiated. ");
                return;
            }
            
        }
        for(IVehicle vehicle : vehicles) {
            vehicle.go();
        }
    }
}

In [ ]:
java VehicleSystem Car Rocket Airplane Car

<img src="img/cat2.jpg"/>

# Dostęp do składowych

* newInstance()
* getMethod(...)
* getMethods()
* getConstructor(...)
* getConstructors()
* getField(...)
* getFields()

In [ ]:
void prettyPrint(Object[] elements){
    for(Object element : elements)
        System.out.println(element);
}

In [ ]:
Class<?> klass = DummyIterator.class;

prettyPrint(klass.getMethods());
prettyPrint(klass.getConstructors());
prettyPrint(klass.getFields());

# Dynamiczna instancjacja

In [ ]:
import java.lang.reflect.*;

Constructor constructor = klass.getConstructor(new Class<?>[0]);
Iterator dummyIterator = (Iterator) constructor.newInstance();

In [ ]:
class DummyIterator implements Iterator {
    private int index = 0;
    
    public DummyIterator(){
    }
    
    public DummyIterator(int index){
        this.index = index;
    }

    @Override
    public boolean hasNext(){
        return false;
    }

    @Override
    public Object next(){
        return null;
    }
}

In [ ]:
Class<?> klass = DummyIterator.class;

Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
iterator dummyIterator = (Iterator) constructor.newInstance(1);

# Dynamiczne wywołanie metody

In [ ]:
Method hasNextMethod = klass.getMethod("hasNext", new Class<?>[0]);
hasNextMethod.getName();
hasNextMethod.getParameterTypes();

In [ ]:
Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);

Method hasNextMethod = klass.getMethod("hasNext", new Class<?>[0]);
hasNextMethod.invoke(dummyIterator);

# Dynamiczny dostęp do pól

In [ ]:
Field indexField = klass.getField("index");
indexField.getName();
indexField.getModifiers();

In [ ]:
Field indexField = klass.getField("index");
|  java.lang.NoSuchFieldException thrown: index
|        at Class.getField (Class.java:1891)
|        at (#300:1)

In [ ]:
Field indexField = klass.getDeclaredField("index");
indexField.getName();
indexField.getModifiers();

In [ ]:
Constructor constructor = klass.getConstructor(new Class<?>[]{int.class});
Iterator dummyIterator = (Iterator) constructor.newInstance(1);

Field indexField = klass.getDeclaredField("index");
indexField.get(dummyIterator);

In [ ]:
indexField.get(dummyIterator);
|  java.lang.IllegalAccessException thrown: class REPL.$JShell$151 cannot access a member of class REPL.$JShell$19D$DummyIterator with modifiers "private"
|        at Reflection.throwIllegalAccessException (Reflection.java:405)
|        at Reflection.throwIllegalAccessException (Reflection.java:396)
|        at Reflection.ensureMemberAccess (Reflection.java:98)
|        at AccessibleObject.slowCheckMemberAccess (AccessibleObject.java:355)
|        at AccessibleObject.checkAccess (AccessibleObject.java:347)
|        at Field.get (Field.java:406)
|        at (#313:1)

In [ ]:
indexField.setAccessible(true);
indexField.get(dummyIterator);
indexField.set(dummyIterator,2);

<img src="img/mouth.jpg"/>

# Dostęp do wszystkich składowych prywatnych

* getDeclaredMethod(...)
* getDeclaredMethods()
* getDeclaredConstructor(...)
* getDeclaredConstructors()
* getDeclaredField(...)
* getDeclaredFields()

Uwaga: składowe dziedziczone nie są zwracane

# Dynamiczne proxy

In [ ]:
import java.lang.reflect.*;

class MethodReplayer implements InvocationHandler {
    private List<String> list = new LinkedList<>();
    private List<Method> invokedMethods = new LinkedList<>();
    private List<Object[]> invokedArguments = new LinkedList<>();

    public Object invoke(Object proxy, Method method, Object[] args) 
            throws Throwable{
        if(!method.getName().equals("toString")){
            invokedMethods.add(method);
            invokedArguments.add(args);
        }
        return method.invoke(list, args);
    }

    public void replay() throws Throwable {
        for(int i = 0; i < invokedMethods.size(); i++){
            invokedMethods.get(i).invoke(list, invokedArguments.get(i));
        }
    }
}

In [ ]:
import static java.lang.reflect.Proxy.newProxyInstance;

MethodReplayer replayer = new MethodReplayer();
List<String> list = (List<String>) newProxyInstance(List.class.getClassLoader(),
        new Class[] { List.class },
        replayer);

list.add("dog");
list.add("cat");
System.out.println(list);

replayer.replay();
System.out.println(list);

<img src="img/cat3.jpg"/>

# Uchwyty do metod

* w API refleksji sygnatura argumentów metody jest wskazywana przez `Class[]`
* w nowym API mamy nowy typ `MethodType`:
  * brak nazwy metody
  * brak typu adresata
  * tylko typy argumentów oraz typ zwracanej wartości

In [ ]:
import java.lang.invoke.*;
import java.lang.invoke.MethodHandles.Lookup;

// np. String toString()
MethodType toStringType = MethodType.methodType(String.class);

// np. Integer parseInt(String str)
MethodType fromStringToInt = MethodType.methodType(Integer.class, String.class);

In [ ]:
class CallMeMaybe {
    public String toString(){
        return "call me maybe";
    }
    
    public Lookup getLookup(){
        return MethodHandles.lookup();
    }
}

In [ ]:
CallMeMaybe maybe = new CallMeMaybe();
Lookup maybeLookup = maybe.getLookup();

MethodHandle maybeHandle = maybeLookup.findVirtual(CallMeMaybe.class, 
    "toString", toStringType);
maybeHandle.invoke(maybe);

<img src="img/ruby-horizontal.svg"/>

# Pomoc wbudowana w REPLa

In [ ]:
[1] pry(main)> require 'hanami/model'
=> true
[2] pry(main)> ls Hanami::Entity
constants: ClassMethods  Schema  Types
Hanami::Entity.methods: inherited
Hanami::Entity#methods: ==  freeze  hash  id  method_missing  to_h  to_hash
[3] pry(main)>

# Informacja o typie

In [ ]:
#!ruby
puts 1.class
puts "Ala ma kota".class

# Weryfikacja przynależności do typu

In [ ]:
object = "Ala ma kota"
if(object.is_a?(String))
    puts "Obiekt jest lancuchem znakow"
end

In [ ]:
# encoding: utf-8
object = "Ala ma kota"
if(String === object)
    puts "Obiekt jest łańcuchem znaków"
end

In [ ]:
# encoding: utf-8
object = [1,2,3]
case(object)
when String
  puts "Obiekt jest łańuchem znaków"
when Array
  puts "Obiekt jest tablicą"
end

# Informacje o hierarchii klas

In [ ]:
class Animal
end

class Mammal < Animal
end

class Cat < Mammal
end

puts Cat.new.class.superclass

In [ ]:
def ancestors(klass, chain=[])
  klass.superclass ? ancestors(klass.superclass, chain + [klass]) : chain.reverse
end

puts ancestors(Bat)

In [ ]:
class Whale < Mammal
end

class Reptile < Animal
end

class Snake < Reptile
end

In [ ]:
def commonAncestor(klass1, klass2)
  ancestors(klass1).zip(ancestors(klass2)).each_cons(2) do |(grandparent1, grandparent2), (parent1, parent2)|
    return grandparent1 if parent1 != parent2 
  end
end

In [ ]:
puts commonAncestor(Bat, Whale)
puts commonAncestor(Snake, Bat)
puts commonAncestor(1.class, Bat)

# Dynamiczne ładowanie klas

In [ ]:
class Car
  def go
    puts "Brum brum"
  end
end

class Airplane
  def go
    puts "Bzzzzz"
  end
end

class Rocket
  def go
    puts "BUUUUUUM"
  end
end

In [ ]:
["Car", "Car", "Rocket", "Airplane"].each do |classname|
  Class.const_get(classname).new.go
end

# Refleksja

In [ ]:
puts Car.new.public_methods.sort

In [ ]:
puts (Car.new.public_methods - 1.public_methods).sort

In [ ]:
class Bike
  def initialize
    @speed = 0
  end

  def go
    @speed = 10
  end

  def stop
    @speed = 0
  end

  def riding?
    @speed > 0
  end

  def instance_variable_set(name, value)
    raise "Don't do it!"
  end
end

In [ ]:
puts Bike.new.instance_variables

# Dynamiczne wywoływanie metod

In [ ]:
bike = Bike.new
puts bike.public_send("riding?")
bike.public_send("go")
puts bike.public_send("riding?")

In [ ]:
bike = Bike.new
#bike.public_send("initialize")
bike.send("initialize")

In [ ]:
bike = Bike.new
bike.send("initialize")

# Dynamiczny dostęp do pól

In [ ]:
bike = Bike.new
puts bike.instance_variable_get("@speed")
bike.instance_variable_set("@speed", 10)
puts bike.instance_variable_get("@speed")

# Dynamiczne proxy

In [ ]:
class Wrapper
  def initialize(object)
    @object = object
    @invocations = []
  end

  def method_missing(*arguments)
    @invocations << arguments 
    @object.public_send(*arguments)
  end

  def replay
    @invocations.each do |arguments|
      @object.public_send(*arguments)
    end
  end
end

In [ ]:
wrapper = Wrapper.new([])
wrapper << 1
wrapper << 2
wrapper << 3
puts wrapper.to_a.join(", ")

wrapper.replay

puts wrapper.to_a.join(", ")

![Pytania? ](http://cliparts.co/cliparts/qcB/jqg/qcBjqgxc5.jpg)